In [3]:
get_ipython().system('pip install scikit-learn seaborn')

from sklearn.metrics import f1_score
from scipy.io import arff

get_ipython().system('pip install gurobipy==11.0.2')
import gurobipy as gp
from gurobipy import *

import os
os.environ['GRB_LICENSE_FILE'] = '/home/hexiaoyu/JointProblem/gurobi.lic'  
import numpy as np
import pandas as pd
from copy import deepcopy
from pyomo.environ import *
import time
import sys
import os
import random

Defaulting to user installation because normal site-packages is not writeable
  Using cached scipy-1.10.1-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.5 MB)
  Using cached pandas-2.0.3-cp38-cp38-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
  Using cached matplotlib-3.7.5-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (9.2 MB)
ERROR: matplotlib 3.7.5 has requirement numpy<2,>=1.20, but you'll have numpy 1.18.3 which is incompatible.
ERROR: scipy 1.10.1 has requirement numpy<1.27.0,>=1.19.5, but you'll have numpy 1.18.3 which is incompatible.
ERROR: pandas 2.0.3 has requirement numpy>=1.20.3; python_version < "3.10", but you'll have numpy 1.18.3 which is incompatible.
ERROR: seaborn 0.13.2 has requirement numpy!=1.24.0,>=1.20, but you'll have numpy 1.18.3 which is incompatible.
Defaulting to user installation because normal site-packages is not writeable


In [4]:
def ind_function(X, label, UB0, UB1, M, T,reg):
    """Fit Estimator based on NCPOP Regressor model and predict y or produce residuals."""
    
    e = gp.Env()
    e.setParam('TimeLimit', 60)
    model = gp.Model(env=e)
    T -= 1  
    I = len(X)  

    # Var
    L = model.addVars(I, name="L", vtype='B')  # Indicator

    f0 = model.addVars(M, T, name="f0", vtype='C')
    phi0 = model.addVars(UB0, (T + 1), name="phi0", vtype='C')
    G0 = model.addVars(UB0, UB0, name="G0", vtype='C')
    F0 = model.addVars(M, UB0, name="F0", vtype='C')
    q0 = model.addVars(UB0, T, name="q0", vtype='C')
    p0 = model.addVars(M, T, name="p0", vtype='C')
    quatr0 = model.addVars(M, (T + 1), name="quatr0", vtype='C')
    quatr_hidden0 = model.addVars(UB0, T, name="quatr_hidden0", vtype='C')

    f1 = model.addVars(M, T, name="f1", vtype='C')
    phi1 = model.addVars(UB1, (T + 1), name="phi1", vtype='C')
    G1 = model.addVars(UB1, UB1, name="G1", vtype='C')
    F1 = model.addVars(M, UB1, name="F1", vtype='C')
    q1 = model.addVars(UB1, T, name="q1", vtype='C')
    p1 = model.addVars(M, T, name="p1", vtype='C')
    quatr1 = model.addVars(M, (T + 1), name="quatr1", vtype='C')
    quatr_hidden1 = model.addVars(UB1, T, name="quatr_hidden1", vtype='C')
    z0_squared = model.addVars(I, T, M, name="z0_squared", vtype='C')
    z1_squared = model.addVars(I, T, M, name="z1_squared", vtype='C')
    

    # Obj
    obj = gp.quicksum(
        (1 - L[i]) * z0_squared[i, t, m] + L[i] * z1_squared[i, t, m]
        for m in range(M) for t in range(T) for i in range(I)
    )
    obj += gp.quicksum(reg * q0[n0, t] ** 2 for n0 in range(UB0) for t in range(T))
    obj += gp.quicksum(reg * q1[n1, t] ** 2 for n1 in range(UB1) for t in range(T))
    obj += gp.quicksum(reg * p0[m, t] ** 2 + reg * p1[m, t] ** 2 for m in range(M) for t in range(T))

    model.setObjective(obj, GRB.MINIMIZE)

    # Constraint 
    model.addConstrs(z0_squared[i, t, m] == (X[i, t, m] - f0[m, t])**2 for i in range(I) for t in range(T) for m in range(M))
    model.addConstrs(z1_squared[i, t, m] == (X[i, t, m] - f1[m, t])**2 for i in range(I) for t in range(T) for m in range(M))
    
    for t in range(T):
        for m in range(M):
            model.addConstr(gp.quicksum(F0[m, n] * phi0[n, t+1] for n in range(UB0)) == quatr0[m, t+1]) 
            model.addConstr(gp.quicksum(F1[m, n] * phi1[n, t+1] for n in range(UB1)) == quatr1[m, t+1])
    model.addConstr(gp.quicksum(f0[m, t] - quatr0[m,(t+1)] - p0[m, t]for t in range(T) for m in range(M)) == 0)
    model.addConstr(gp.quicksum(f1[m, t] - quatr1[m,(t+1)] - p1[m, t]for t in range(T) for m in range(M)) == 0)
    for t in range(T):
        for n in range(UB0):
            model.addConstr(gp.quicksum(G0[n, nn] * phi0[nn, t] for nn in range(UB0)) == quatr_hidden0[n, t])
        for n in range(UB1):
            model.addConstr(gp.quicksum(G1[n, nn] * phi1[nn, t] for nn in range(UB1)) == quatr_hidden1[n, t])
    model.addConstr(gp.quicksum(phi0[n, t+1] - quatr_hidden0[n, t]- q0[n, t] for t in range(T) for n in range(UB0))  == 0)
    model.addConstr(gp.quicksum(phi1[n, t+1] - quatr_hidden1[n, t]- q1[n, t] for t in range(T) for n in range(UB1))  == 0)
    
    model.update()
    model.setParam("MIPGap", 1e-6)  # 收敛精度
    model.Params.NonConvex = 2
    model.setParam('IntFeasTol', 1e-9)
    model.optimize()

    if model.status == GRB.Status.OPTIMAL:
        print("THIS IS OPTIMAL SOLUTION")
    else:
        print("THIS IS NOT OPTIMAL SOLUTION")

    # 获取结果
    data_dict = {
        'G0': [model.getAttr("x", G0)[h, k] for (h, k) in model.getAttr("x", G0)],
        'F0': [model.getAttr("x", F0)[h, k] for (h, k) in model.getAttr("x", F0)],
        'G1': [model.getAttr("x", G1)[h, k] for (h, k) in model.getAttr("x", G1)],
        'F1': [model.getAttr("x", F1)[h, k] for (h, k) in model.getAttr("x", F1)]
    }

    label_out = np.array([round(model.getAttr("x", L)[h]) for h in model.getAttr("x", L)])

    print(f'M={M}, T={T}')
    return model, label_out, data_dict

In [5]:
def system_matrix(arr_G, arr_F, M, UB):
    threshold=0.001
    arr_rounded_G = np.round(arr_G, decimals=3)
    arr_rounded_F = np.round(arr_F, decimals=3)
    arr_rounded_G[np.abs(arr_rounded_G) < threshold] = 0 
    arr_rounded_F[np.abs(arr_rounded_F) < threshold] = 0 
    rows_G, cols_G = arr_G.shape
    rows_F, cols_F = arr_F.shape

    if not arr_rounded_G.any() and not arr_rounded_F.any():
        print("Warning: arr_rounded_G is empty or contains only zeros.")
        return arr_G, arr_F,0  
    
    else:
        nonzero_rows_G, nonzero_cols_G = np.nonzero(arr_rounded_G.reshape(UB, UB)) 
        nonzero_rows_F, nonzero_cols_F = np.nonzero(arr_rounded_F.reshape(M,UB))
        def max_max(G):
            if G.size ==0: 
                return 0
            else:
                return max(G)
        N = max(max_max(nonzero_rows_G), max_max(nonzero_cols_G), max_max(nonzero_cols_F)) + 1
        # print(f"N:{N}")
        system_G = arr_G[:N, :N]
        system_F = arr_F[:, :N]
        return system_G, system_F, N
        
def IF_test(X_data,X_label,UB0,UB1,name,win_size,seed, reg):
  """
  X_data.shape: N,T,M, for ecg, X_data contains all the data
  X_lebel.shape: N     for both, X_label is like [1,1,1,0,0,0]
  name: 'lds' or 'ecg'
  """
  np.random.seed(seed)

  if name == 'lds':
    data_in = X_data

  elif name == 'ecg':
    X_1 = X_data[X_data.target==b'1'].iloc[:,:-1].values
    X_2 = X_data[X_data.target==b'2'].iloc[:,:-1].values
    idx_1 = np.arange(len(X_1))
    idx_2 = np.arange(len(X_2))
    iidx_1 = random.sample(sorted(idx_1), 15)
    iidx_2 = random.sample(sorted(idx_2), 15)
    np.random.shuffle(iidx_1)
    np.random.shuffle(iidx_2)
    XX_1 = X_1[iidx_1]
    XX_2 = X_2[iidx_2]
    data_in = np.concatenate((XX_1, XX_2), axis=0).reshape(30,140,1)

    if win_size == 140:
      data_in = data_in
    else:
      s = random.sample(sorted(np.arange(140-win_size)),1)
      s = s[0]
      data_in = data_in[:,s:(s+win_size),:]
    print(data_in.shape)

  I = np.random.randint(1, len(data_in) + 1, size=1)[0] 
  I_idx = np.random.choice(len(data_in), size=I, replace=False)
  data_ = data_in[I_idx, :, :]

  label_in = X_label[I_idx]
  print("lds initialized label:", label_in)

  m = data_in.shape[2]
  t = data_in.shape[1]
  print("m,t:",m,t)
  model, label_out, data_dict = ind_function(X=data_,label=label_in, UB0=UB0,UB1=UB1,M=m,T=t, reg=reg)
  G0 = np.array(np.array(data_dict['G0']).reshape(UB0,UB0))
  F0 = np.array(np.array(data_dict['F0']).reshape(m,UB0))
  G1 = np.array(np.array(data_dict['G1']).reshape(UB1,UB1))
  F1 = np.array(np.array(data_dict['F1']).reshape(m,UB1))
  G0, F0, N0=system_matrix(G0, F0, m, UB=UB0)
  G1, F1, N1=system_matrix(G1, F1, m, UB=UB1)
  print(f"Hidden_State Dimension of System 1 is:{N0}.")
  print(f"Hidden_State Dimension of System 2 is:{N1}.")
  print(f"System matrices are G0: \n {G0} and \n F0:\n{F0}.")
  print(f"System matrices are G1: \n {G1} and \n F1:\n{F1}.")

  f1_1 = f1_score(label_in, label_out)
  f1_2 = f1_score(label_in, 1-label_out)

  if f1_1>f1_2:
      f1 = f1_1
  else:
      f1 = f1_2
  
  if np.sum(np.array(data_dict['G0']) == 0) >= (N0*N0+N0*m)/3 or np.sum(np.array(data_dict['F0']) == 0) >= (N0*N0+N0*m)/3 or np.sum(np.array(data_dict['G1']) == 0) >= (N1*N1+N1*m)/3 or np.sum(np.array(data_dict['F1']) == 0) >= (N1*N1+N1*m)/3 or f1==0:
     print("The System Matrices Is Sparse.")
     validation = 0
  else:
     print("The System Matrices Is Dense!")
     validation = 1
  print('f1:',f1)

  return f1,validation

In [4]:
def reg_open_npy(folder_path):
    '''Read all the files under folder path
    '''
    
    data_dict = {
        "countvalidation": {},
        "std": {},
        "mean": {},
        "validation": {},
        "duration": {}
    }

    pattern = re.compile(r'_(\d+(?:\.\d+)?)\.npy$')


    for filename in os.listdir(folder_path):
        if filename.endswith(".npy"):
            file_path = os.path.join(folder_path, filename)

            match = pattern.search(filename)
            if match:
                key = match.group(1)
                if "validation" in filename:
                    data_dict["validation"][key] = np.load(file_path)
                    data_dict["countvalidation"][key] = sum(1 for x in data_dict["validation"][key] if x != 0)
                elif "std" in filename:
                    data_dict["std"][key] = np.load(file_path)
                elif "mean" in filename:
                    data_dict["mean"][key] = np.load(file_path)
                elif "duration" in filename:
                    data_dict["duration"][key] = np.load(file_path)

    for category in data_dict:
        data_dict[category] = dict(sorted(data_dict[category].items()))

    for category, values in data_dict.items():
        print(f"\nCategory: {category} ")
        for key, data in values.items():
            print(f"{key}: {data}")


def test(data, data_list,I,T,N,reg):
    f1_if_Gurobi_lds = []  # 5*10*3
    duration_if_Gurobi_lds = []
    f1_if_Gurobi_validation = []
    for n,data in enumerate(data_list):
        X_1 = data[:N*I].reshape(N,I,20,2)[:,:,:T,:]
        X_2 = data[(320-N*I):].reshape(N,I,20,2)[:,:,:T,:] 

        t_start = time.time()

        for i in range(N):
            xx_1 = X_1[i]
            xx_2 = X_2[i]
            xx_all = np.concatenate((xx_1,xx_2),axis=0)
            label = np.concatenate((np.zeros(I),np.ones(I)),axis=0)
            for j in range(5):
                print("n:",n+2)
                print("regularization:",reg)
                f1,validation = IF_test(xx_all,label,5,5,'lds',20,(30+i+j), reg) 
                print(f1)
                f1_if_Gurobi_lds.append(f1)
                f1_if_Gurobi_validation.append(validation)
        duration_if_Gurobi_lds.append(time.time()-t_start)
    return f1_if_Gurobi_lds, duration_if_Gurobi_lds, f1_if_Gurobi_validation


if __name__ == '__main__':
    path_list = ['./dataset/2_2_test.npy', './dataset/3_2_test.npy','./dataset/4_2_test.npy']
    data_list = []
    label_list = []
    for path in path_list:
        data = np.load(path)
        T = 20
        M = 2
        X_data = data.reshape(-1,T,M)
        X_label = np.concatenate((np.zeros(16),np.ones(16)),axis=0)
        print(X_data.shape, X_label)
        data_list.append(X_data)
        label_list.append(X_label)

    small_values = [0.05, 0.5, 5]
    large_values = list(range(650, 1000, 50))
    values = np.array(large_values)
    for j in values:
        I=10
        T=3
        N=10
        f1_if_Gurobi_lds, duration_if_Gurobi_lds, f1_if_Gurobi_validation = test(data, data_list,I, T, N, j)
        f1_if_Gurobi_lds = np.array(f1_if_Gurobi_lds).reshape(3,50) 
        f1_if_Gurobi_lds_mean = np.mean(f1_if_Gurobi_lds, axis=1)
        f1_if_Gurobi_lds_std = np.std(f1_if_Gurobi_lds, axis=1)

        with open(f'./Result-Synthetic/f1_if_Gurobi_lds_mean_{j}.npy', 'wb') as f:
            np.save(f, f1_if_Gurobi_lds_mean)
        with open(f'./Result-Synthetic/f1_if_Gurobi_lds_std_{j}.npy', 'wb') as f:
            np.save(f, f1_if_Gurobi_lds_std)
        with open(f'./Result-Synthetic/duration_if_Gurobi_lds_{j}.npy', 'wb') as f:
            np.save(f, duration_if_Gurobi_lds)
        with open(f'./Result-Synthetic/validation_if_Gurobi_lds_{j}.npy', 'wb') as f:
            np.save(f, f1_if_Gurobi_validation)
        folder_path = "/home/hexiaoyu/JointProblem/Result-Synthetic/"


Category: countvalidation 
0.5: 0
200: 10
250: 10
270: 13
300: 9
350: 9
400: 9
450: 14
470: 12
5.0: 2
520: 8
550: 11
620: 11
800: 11
820: 10
920: 10

Category: std 
0.5: [0.30509245 0.3        0.29503731]
200: [0.11843654 0.0485904  0.15960979]
250: [0.11928459 0.0587296  0.15624308]
270: [0.1157522  0.07291815 0.15734876]
300: [0.137025   0.06456098 0.15281721]
350: [0.11394959 0.05205141 0.15091288]
400: [0.11553801 0.05205141 0.15667815]
450: [0.11553801 0.05205141 0.15667815]
470: [0.12741638 0.05054146 0.1461098 ]
5.0: [0.30509245 0.3        0.26792391]
520: [0.13473368 0.05195322 0.15962454]
550: [0.13081892 0.06741661 0.15667815]
620: [0.11553801 0.05205141 0.15667815]
800: [0.11641023 0.05205141 0.15667815]
820: [0.11581928 0.05205141 0.15667815]
920: [0.11553801 0.05205141 0.15667815]

Category: mean 
0.5: [0.75603175 0.9        0.77269841]
200: [0.88152679 0.97613527 0.84196054]
250: [0.87875014 0.96989596 0.84506788]
270: [0.88450446 0.96769079 0.84290915]
300: [0.87214219 